# Holography Simulation using ML
## Simons Observatory Large Aperture Telescope
### Grace E. Chesmore and Jeff McMahon - McMahonCosmologyGroup

In [ ]:
import numpy as np
import tele_geo as tg
import ap_field as af
import pan_mod as pm
import optics_analyze as oa
import far_field as ff
import ap_fitting as afit

# Machine Learning packages
import sklearn
from sklearn.linear_model import LinearRegression
import pickle

# Plot settings
#%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

# To get Times font on this computer:
font_manager.fontManager.addfont(
    "/home/chesmore/.local/share/fonts/times-new-roman.ttf"
)
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams["font.size"] = 28
plt.rcParams["image.cmap"] = "magma"
plt.style.use("ggplot")
plt.rcParams["axes.unicode_minus"] = False

# Singular and Binocular positions
rx_x = np.array([0, 0, 0])
rx_z = np.array([-600 * (3 / 2), 600 * (3 / 2), 0])
el = np.array([oa.el_offset(-600 * (3 / 2)), oa.el_offset(600 * (3 / 2)), 0])
az = np.array([0, 0, 0])
shift_A = ["y", oa.sh_z(rx_z[0])]
shift_B = ["y", oa.sh_z(rx_z[1])]
shift_C = ["y", oa.sh_z(rx_z[2])]

# Trinocular positions
rx_x_tri = np.array([-519.62 * (3 / 2), 519.62 * (3 / 2), 0])
rx_z_tri = np.array([-300 * (3 / 2), -300 * (3 / 2), 600 * (3 / 2)])
el_tri = np.array(
    [oa.el_offset(rx_z_tri[0]), oa.el_offset(rx_z_tri[1]), oa.el_offset(rx_z_tri[2])]
)
az_tri = np.array(
    [oa.az_offset(rx_x_tri[0]), oa.az_offset(rx_x_tri[1]), oa.az_offset(rx_x_tri[2])]
)

shift_A_tri = ["xy", oa.sh_x(rx_x_tri[0]), oa.sh_z(rx_z_tri[0])]
shift_B_tri = ["xy", oa.sh_x(rx_x_tri[1]), oa.sh_z(rx_z_tri[1])]
shift_C_tri = ["y", oa.sh_z(rx_z_tri[2])]

## Holography Measurement
Simulate three holography measurements, at three different spots on the focal plane. These will then be fit using the ML models either with a singular holography measurement, or with a binocular holography measurement. We also compute the aberrations at all measurement locations on the focal plane (aperture fields with no adjuster offsets). These will be subtracted from the measurements with panel errors, leaving only the pathlength differences due to adjuster offsets.

In [ ]:
# Define FOV of receiver feed (RX) positions 
# (i.e. define direction of outgoing rays from the RX).
tele_geo = tg.initialize_telescope_geometry()
th = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, tele_geo.N_scan)
ph = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, tele_geo.N_scan)

# Define adjuster offsets, as random distribution on the micron scale
adj_1 = np.random.randn(77*5) * 20 # [um]
adj_2 = np.random.randn(69*5) * 25 # [um]

# Define panels on M1 and M2. Here you can define the
# magnitude of the adjuster offsets on each mirror:
pan_mod_m2 = pm.panel_model_from_adjuster_offsets(
    2, adj_2, 1,0
)  # Panel Model on M2
pan_mod_m1 = pm.panel_model_from_adjuster_offsets(
    1, adj_1, 1,0
)  # Panel Model on M1

rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
tele_geo = tg.tele_geo_init(rx1[0],rx1[1],rx1[2],el[0],az[0])
rxmirror_A = af.ray_mirror_pts(rx1, tele_geo, th, ph)
dat_A = afit.take_measurement(np.zeros(77*5),np.zeros(77*5),0,tele_geo,rxmirror_A)
dat_A = np.loadtxt(dat_A)
x_A, y_A, meas_A, ampl_A, geo = afit.analyze_holography(
    dat_A, tele_geo, 0, 1, 0,shift_A
)
meas_A = np.where((abs(ampl_A)/np.max(abs(ampl_A)))>=.3,meas_A-np.mean(meas_A),0)

out1 = af.aperature_fields_from_panel_model(
    pan_mod_m1, pan_mod_m2, rx1, tele_geo, th, ph, rxmirror_A
) # apert. fields

beam1 = ff.far_field_sim(out1, tele_geo, rx1) # far field beam
amp1 = 20 * np.log10(abs(beam1[2, :]) / np.max(abs(beam1[2, :]))) # far field beam amplitude [dB]

rx2 = np.array([rx_x[1], 209.09, rx_z[1]])
tele_geo = tg.tele_geo_init(rx2[0],rx2[1],rx2[2],el[1],az[1])
rxmirror_B = af.ray_mirror_pts(rx2, tele_geo, th, ph)
dat_B = afit.take_measurement(np.zeros(77*5),np.zeros(77*5),0,tele_geo,rxmirror_B)
dat_B = np.loadtxt(dat_B)
x_B, y_B, meas_B, ampl_B, geo = afit.analyze_holography(
    dat_B, tele_geo, 0, 1, 0,shift_B
)
meas_B = np.where((abs(ampl_B)/np.max(abs(ampl_B)))>=.3,meas_B-np.mean(meas_B),0)

out2 = af.aperature_fields_from_panel_model(
    pan_mod_m1, pan_mod_m2, rx2, tele_geo, th, ph, rxmirror_B
) # apert. fields

beam2 = ff.far_field_sim(out2, tele_geo, rx2) # far field beam
amp2 = 20 * np.log10(abs(beam2[2, :]) / np.max(abs(beam2[2, :]))) # far field beam amplitude [dB]

rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
tele_geo = tg.tele_geo_init(rx3[0],rx3[1],rx3[2],el[2],az[2])
rxmirror_C = af.ray_mirror_pts(rx3, tele_geo, th, ph)
dat_C = afit.take_measurement(np.zeros(77*5),np.zeros(77*5),0,tele_geo,rxmirror_C)
dat_C = np.loadtxt(dat_C)
x_C, y_C, meas_C, ampl_C, geo = afit.analyze_holography(
    dat_C, tele_geo, 0, 1, 0,shift_C
)
meas_C = np.where((abs(ampl_C)/np.max(abs(ampl_C)))>=.3,meas_C-np.mean(meas_C),0)

out3 = af.aperature_fields_from_panel_model(
    pan_mod_m1, pan_mod_m2, rx3, tele_geo, th, ph, rxmirror_C
) # apert. fields

beam3 = ff.far_field_sim(out3, tele_geo, rx3) # far field beam
amp3 = 20 * np.log10(abs(beam3[2, :]) / np.max(abs(beam3[2, :]))) # far field beam amplitude [dB]

In [ ]:
rx1_tri = np.array([rx_x_tri[0], 209.09, rx_z_tri[0]])
tele_geo = tg.tele_geo_init(rx1_tri[0], rx1_tri[1], rx1_tri[2], el_tri[0], az_tri[0])
rxmirror_A_tri = af.ray_mirror_pts(rx1_tri, tele_geo, th, ph)
dat_A = afit.take_measurement(
    np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_A_tri
)
dat_A = np.loadtxt(dat_A)
x_A_tri, y_A_tri, meas_A_tri, ampl_A_tri, geo = afit.analyze_holography(
    dat_A, tele_geo, 0, 1, 0, shift_A_tri
)
meas_A_tri = np.where(
    (abs(ampl_A_tri) / np.max(abs(ampl_A_tri))) >= 0.3,
    meas_A_tri - np.mean(meas_A_tri),
    0,
)

out1_tri = af.aperature_fields_from_panel_model(
    pan_mod_m1, pan_mod_m2, rx1_tri, tele_geo, th, ph, rxmirror_A_tri
)  # apert. fields

beam1_tri = ff.far_field_sim(out1_tri, tele_geo, rx1_tri)  # far field beam
amp1_tri = 20 * np.log10(
    abs(beam1_tri[2, :]) / np.max(abs(beam1_tri[2, :]))
)  # far field beam amplitude [dB]

rx2_tri = np.array([rx_x_tri[1], 209.09, rx_z_tri[1]])
tele_geo = tg.tele_geo_init(rx2_tri[0], rx2_tri[1], rx2_tri[2], el_tri[1], az_tri[1])
rxmirror_B_tri = af.ray_mirror_pts(rx2_tri, tele_geo, th, ph)
dat_B = afit.take_measurement(
    np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_B_tri
)
dat_B = np.loadtxt(dat_B)
x_B_tri, y_B_tri, meas_B_tri, ampl_B_tri, geo = afit.analyze_holography(
    dat_B, tele_geo, 0, 1, 0, shift_B_tri
)
meas_B_tri = np.where(
    (abs(ampl_B_tri) / np.max(abs(ampl_B_tri))) >= 0.3,
    meas_B_tri - np.mean(meas_B_tri),
    0,
)

out2_tri = af.aperature_fields_from_panel_model(
    pan_mod_m1, pan_mod_m2, rx2_tri, tele_geo, th, ph, rxmirror_B_tri
)  # apert. fields

beam2_tri = ff.far_field_sim(out2_tri, tele_geo, rx2_tri)  # far field beam
amp2_tri = 20 * np.log10(
    abs(beam2_tri[2, :]) / np.max(abs(beam2_tri[2, :]))
)  # far field beam amplitude [dB]

rx3_tri = np.array([rx_x_tri[2], 209.09, rx_z_tri[2]])
tele_geo = tg.tele_geo_init(rx3_tri[0], rx3_tri[1], rx3_tri[2], el_tri[2], az_tri[2])
rxmirror_C_tri = af.ray_mirror_pts(rx3_tri, tele_geo, th, ph)
dat_C = afit.take_measurement(
    np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_C_tri
)
dat_C = np.loadtxt(dat_C)
x_C_tri, y_C_tri, meas_C_tri, ampl_C_tri, geo = afit.analyze_holography(
    dat_C, tele_geo, 0, 1, 0, shift_C_tri
)
meas_C_tri = np.where(
    (abs(ampl_C_tri) / np.max(abs(ampl_C_tri))) >= 0.3,
    meas_C_tri - np.mean(meas_C_tri),
    0,
)

out3_tri = af.aperature_fields_from_panel_model(
    pan_mod_m1, pan_mod_m2, rx3_tri, tele_geo, th, ph, rxmirror_C_tri
)  # apert. fields

beam3_tri = ff.far_field_sim(out3_tri, tele_geo, rx3_tri)  # far field beam
amp3_tri = 20 * np.log10(
    abs(beam3_tri[2, :]) / np.max(abs(beam3_tri[2, :]))
)  # far field beam amplitude [dB]

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(14, 3))
plt.suptitle(
    "Far-field Measurements - Singular & Binocular", x=0.44, y=1.1, fontsize=30
)
axes[0].set_ylabel("el. [arcmin]")
axes[0].set_xlabel(r"az. [arcmin]")
axes[1].set_xlabel(r"az. [arcmin]")
axes[2].set_xlabel(r"az. [arcmin]")
first = axes[0].scatter(
    np.real(beam1[0, :]) * (180 / np.pi) * 60,
    np.real(beam1[1, :]) * (180 / np.pi) * 60,
    c=amp1,
    vmin=-60,
)
first = axes[2].scatter(
    np.real(beam2[0, :]) * (180 / np.pi) * 60,
    np.real(beam2[1, :]) * (180 / np.pi) * 60,
    c=amp2,
    vmin=-60,
)
first = axes[1].scatter(
    np.real(beam3[0, :]) * (180 / np.pi) * 60,
    np.real(beam3[1, :]) * (180 / np.pi) * 60,
    c=amp3,
    vmin=-60,
)

axes[0].axis("equal")
axes[1].axis("equal")
axes[2].axis("equal")
f.colorbar(first, ax=axes, label="dB")
plt.show()

np.savetxt(
    "/data/chesmore/sim_out/rx_" + str(rx1) + "_holog.txt",
    np.c_[
        np.real(beam1[0, :]),
        np.real(beam1[1, :]),
        np.real(beam1[2, :]),
        np.imag(beam1[2, :]),
    ],
)
np.savetxt(
    "/data/chesmore/sim_out/rx_" + str(rx2) + "_holog.txt",
    np.c_[
        np.real(beam2[0, :]),
        np.real(beam2[1, :]),
        np.real(beam2[2, :]),
        np.imag(beam2[2, :]),
    ],
)
np.savetxt(
    "/data/chesmore/sim_out/rx_" + str(rx3) + "_holog.txt",
    np.c_[
        np.real(beam3[0, :]),
        np.real(beam3[1, :]),
        np.real(beam3[2, :]),
        np.imag(beam3[2, :]),
    ],
)

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(14, 3))
plt.suptitle("Far-field Measurements - Trinocular", x=0.44, y=1.1, fontsize=30)
axes[0].set_ylabel("el. [arcmin]")
axes[0].set_xlabel(r"az. [arcmin]")
axes[1].set_xlabel(r"az. [arcmin]")
axes[2].set_xlabel(r"az. [arcmin]")
first = axes[0].scatter(
    np.real(beam1_tri[0, :]) * (180 / np.pi) * 60,
    np.real(beam1_tri[1, :]) * (180 / np.pi) * 60,
    c=amp1,
    vmin=-60,
)
first = axes[2].scatter(
    np.real(beam2_tri[0, :]) * (180 / np.pi) * 60,
    np.real(beam2_tri[1, :]) * (180 / np.pi) * 60,
    c=amp2,
    vmin=-60,
)
first = axes[1].scatter(
    np.real(beam3_tri[0, :]) * (180 / np.pi) * 60,
    np.real(beam3_tri[1, :]) * (180 / np.pi) * 60,
    c=amp3,
    vmin=-60,
)

axes[0].axis("equal")
axes[1].axis("equal")
axes[2].axis("equal")

f.colorbar(first, ax=axes, label="dB")
plt.show()

np.savetxt(
    "/data/chesmore/sim_out/rx_" + str(rx1_tri) + "_holog_tri.txt",
    np.c_[
        np.real(beam1_tri[0, :]),
        np.real(beam1_tri[1, :]),
        np.real(beam1_tri[2, :]),
        np.imag(beam1_tri[2, :]),
    ],
)
np.savetxt(
    "/data/chesmore/sim_out/rx_" + str(rx2_tri) + "_holog_tri.txt",
    np.c_[
        np.real(beam2_tri[0, :]),
        np.real(beam2_tri[1, :]),
        np.real(beam2_tri[2, :]),
        np.imag(beam2_tri[2, :]),
    ],
)
np.savetxt(
    "/data/chesmore/sim_out/rx_" + str(rx3_tri) + "_holog_tri.txt",
    np.c_[
        np.real(beam3_tri[0, :]),
        np.real(beam3_tri[1, :]),
        np.real(beam3_tri[2, :]),
        np.imag(beam3_tri[2, :]),
    ],
)

## Aperture Fields
Here we read in a holography measurement. From the far-field beam measurements, we perform a discrete fourier transform, yielding the aperture fields (i.e. the amplitude and phase of the beam at the aperture plane). 

In [ ]:
rx1 = np.array([rx_x[0], 209.09, rx_z[0]])
tele_geo = tg.tele_geo_init(rx1[0], rx1[1], rx1[2], el[0], az[0])
dat_A = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx1) + "_holog.txt")
x_A, y_A, phase_A, ampl_A, geo = afit.analyze_holography(
    dat_A, tele_geo, 0, 1, 0, shift_A
)

rx2 = np.array([rx_x[1], 209.09, rx_z[1]])
tele_geo = tg.tele_geo_init(rx2[0], rx2[1], rx2[2], el[1], az[1])
dat_B = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx2) + "_holog.txt")
x_B, y_B, phase_B, ampl_B, geo = afit.analyze_holography(
    dat_B, tele_geo, 0, 1, 0, shift_B
)

rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
tele_geo = tg.tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])
dat_C = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx3) + "_holog.txt")
x_C, y_C, phase_C, ampl_C, geo = afit.analyze_holography(
    dat_C, tele_geo, 0, 1, 0, shift_C
)

phase_A = np.where(
    (abs(ampl_A) / np.max(abs(ampl_A))) >= 0.3, phase_A - np.mean(phase_A), 0
)
phase_B = np.where(
    (abs(ampl_B) / np.max(abs(ampl_B))) >= 0.3, phase_B - np.mean(phase_B), 0
)
phase_C = np.where(
    (abs(ampl_C) / np.max(abs(ampl_C))) >= 0.3, phase_C - np.mean(phase_C), 0
)
phase_A -= meas_A
phase_B -= meas_B
phase_C -= meas_C

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(14, 3.5), sharey=True)
plt.suptitle(
    "Measurement Aperture Fields - Singular & Binocular", fontsize=30, x=0.44, y=1.15
)
colors = axes[0].scatter(x_A, y_A, c=1e6 * phase_A / tele_geo.k, vmin=-100, vmax=100)
axes[0].axis("equal")
axes[1].scatter(x_B, y_B, c=1e6 * phase_B / tele_geo.k, vmin=-100, vmax=100)
axes[1].axis("equal")
axes[2].scatter(x_C, y_C, c=1e6 * phase_C / tele_geo.k, vmin=-100, vmax=100)
axes[2].axis("equal")
axes[0].set_xlabel("x (m)")
axes[0].set_ylabel("y (m)")
axes[1].set_xlabel("x (m)")
axes[2].set_xlabel("x (m)")
axes[0].set_title("(0,0,-900)")
axes[1].set_title("(0,0,900)")
axes[2].set_title("(0,0,0)")
f.colorbar(colors, ax=axes, label=r"$\mu m$")
plt.show()

In [ ]:
tele_geo = tg.tele_geo_init(rx1_tri[0], rx1_tri[1], rx1_tri[2], el_tri[0], az_tri[0])
dat_A = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx1_tri) + "_holog_tri.txt")
x_A_tri, y_A_tri, phase_A_tri, ampl_A_tri, geo = afit.analyze_holography(
    dat_A, tele_geo, 0, 1, 0, shift_A_tri
)

tele_geo = tg.tele_geo_init(rx2_tri[0], rx2_tri[1], rx2_tri[2], el_tri[1], az_tri[1])
dat_B = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx2_tri) + "_holog_tri.txt")
x_B_tri, y_B_tri, phase_B_tri, ampl_B_tri, geo = afit.analyze_holography(
    dat_B, tele_geo, 0, 1, 0, shift_B_tri
)

tele_geo = tg.tele_geo_init(rx3_tri[0], rx3_tri[1], rx3_tri[2], el_tri[2], az_tri[2])
dat_C = np.loadtxt("/data/chesmore/sim_out/rx_" + str(rx3_tri) + "_holog_tri.txt")
x_C_tri, y_C_tri, phase_C_tri, ampl_C_tri, geo = afit.analyze_holography(
    dat_C, tele_geo, 0, 1, 0, shift_C_tri
)

phase_A_tri = np.where(
    (abs(ampl_A_tri) / np.max(abs(ampl_A_tri))) >= 0.3,
    phase_A_tri - np.mean(phase_A_tri),
    0,
)
phase_B_tri = np.where(
    (abs(ampl_B_tri) / np.max(abs(ampl_B_tri))) >= 0.3,
    phase_B_tri - np.mean(phase_B_tri),
    0,
)
phase_C_tri = np.where(
    (abs(ampl_C_tri) / np.max(abs(ampl_C_tri))) >= 0.3,
    phase_C_tri - np.mean(phase_C_tri),
    0,
)
phase_A_tri -= meas_A_tri
phase_B_tri -= meas_B_tri
phase_C_tri -= meas_C_tri

f, axes = plt.subplots(1, 3, figsize=(14, 3.5), sharey=True)
plt.suptitle("Measurement Aperture Fields - Trinocular", fontsize=30, x=0.44, y=1.15)
colors = axes[0].scatter(
    x_A_tri, y_A_tri, c=1e6 * phase_A_tri / tele_geo.k, vmin=-100, vmax=100
)
axes[0].axis("equal")
axes[1].scatter(x_B_tri, y_B_tri, c=1e6 * phase_B_tri / tele_geo.k, vmin=-100, vmax=100)
axes[1].axis("equal")
axes[2].scatter(x_C_tri, y_C_tri, c=1e6 * phase_C_tri / tele_geo.k, vmin=-100, vmax=100)
axes[2].axis("equal")

axes[0].set_xlabel("x (m)")
axes[0].set_ylabel("y (m)")
axes[1].set_xlabel("x (m)")
axes[2].set_xlabel("x (m)")

f.colorbar(colors, ax=axes, label=r"$\mu m$")
plt.show()

## Fitting adjuster offsets with Machine Learning 

In [ ]:
# To load in an external model, run the following line:
trinocular_model = pickle.load(open("../ml-models/model_trinocular.sav", "rb"))
binocular_model = pickle.load(open("../ml-models/model_binocular2.sav", "rb"))
singular_model = pickle.load(open("../ml-models/model_singular1.sav", "rb"))

pathl_meas3 = np.reshape(
    (np.concatenate((phase_A_tri, phase_B_tri, phase_C_tri))),
    (1, len(np.concatenate((phase_A_tri, phase_B_tri, phase_C_tri)))),
)
adj_fit3 = trinocular_model.predict(pathl_meas3)
pathl_meas2 = np.reshape(
    (np.concatenate((phase_A, phase_B))), (1, len(np.concatenate((phase_A, phase_B))))
)
adj_fit2 = binocular_model.predict(pathl_meas2)
pathl_meas1 = np.reshape((phase_C), (1, len(phase_C)))
adj_fit1 = singular_model.predict(pathl_meas1)
adjs_real = np.concatenate((adj_1, adj_2)) / 1e3

## Evaluate the results
Here we evaluate how well the ML model corrected adjuster offsets, for singular, binocular, and trinocular receiver feed positionos. To do so, we pass in the ML guess of adjuster offsets into a function which takes a measurement with specified perturbed mirror surfaces. The plots below show the resulting aperture phase after using the ML fitting.

In [ ]:
adjust = adj_fit1[0]

rx3 = np.array([rx_x[2], 209.09, rx_z[2]])
tele_geo_C = tg.tele_geo_init(rx3[0], rx3[1], rx3[2], el[2], az[2])
phase_C = af.model_of_adj_offs(adjs_real * 1e3, shift_C, tele_geo_C, "total")
phase_fit_C1 = af.model_of_adj_offs(adjust * 1e3, shift_C, tele_geo_C, "total")
phase_tot_new_C1 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "total"
)
phase_m1_new_C1 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m1"
)
phase_m2_new_C1 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m2"
)
phase_m1_model_C1 = af.model_of_adj_offs((adjust * 1e3), shift_C, tele_geo_C, "m1")
phase_m2_model_C1 = af.model_of_adj_offs((adjust * 1e3), shift_C, tele_geo_C, "m2")
phase_m1_C = af.model_of_adj_offs((adjs_real * 1e3), shift_C, tele_geo_C, "m1")
phase_m2_C = af.model_of_adj_offs((adjs_real * 1e3), shift_C, tele_geo_C, "m2")

adjust = adj_fit2[0]
phase_fit_C2 = af.model_of_adj_offs(adjust * 1e3, shift_C, tele_geo_C, "total")
phase_tot_new_C2 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "total"
)
phase_m1_new_C2 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m1"
)
phase_m2_new_C2 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m2"
)
phase_m1_model_C2 = af.model_of_adj_offs((adjust * 1e3), shift_C, tele_geo_C, "m1")
phase_m2_model_C2 = af.model_of_adj_offs((adjust * 1e3), shift_C, tele_geo_C, "m2")

adjust = adj_fit3[0]
phase_fit_C3 = af.model_of_adj_offs(adjust * 1e3, shift_C, tele_geo_C, "total")
phase_tot_new_C3 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "total"
)
phase_m1_new_C3 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m1"
)
phase_m2_new_C3 = af.model_of_adj_offs(
    ((adjs_real - adjust) * 1e3), shift_C, tele_geo_C, "m2"
)
phase_m1_model_C3 = af.model_of_adj_offs((adjust * 1e3), shift_C, tele_geo_C, "m1")
phase_m2_model_C3 = af.model_of_adj_offs((adjust * 1e3), shift_C, tele_geo_C, "m2")

### Plotting aperture fields
Top row: Singular, Middle row: Binocular, Bottom row: Trinocular. First column: only contributions from primary mirror adjuster offsets, Second column: only contributions from secondary mirror adjuster offsets.

In [ ]:
fig, axes = plt.subplots(
    nrows=3,
    ncols=2,
    figsize=(8, 10),
    sharex=True,
    sharey=True,
    subplot_kw={"adjustable": "box"},
)
matplotlib.rcParams["font.size"] = 28
plt.rcParams["image.cmap"] = "magma"
title_font_size = 28

col_range = 50
cc = np.where(x_C ** 2 + y_C ** 2 <= 5 ** 2)

first = axes[0, 0].scatter(
    x_C[cc],
    y_C[cc],
    c=1e6 * (phase_m1_new_C1[cc] - phase_m1_new_C1[cc] - (meas_C[cc])) / tele_geo.k,
    vmin=-col_range,
    vmax=col_range,
)

axes[0, 1].scatter(
    x_C[cc],
    y_C[cc],
    c=1e6 * (phase_m2_new_C1[cc] - (meas_C[cc])) / tele_geo.k,
    vmin=-col_range,
    vmax=col_range,
)

axes[1, 0].scatter(
    x_C[cc],
    y_C[cc],
    c=1e6 * (phase_m1_new_C2[cc] - (meas_C[cc])) / tele_geo.k,
    vmin=-col_range,
    vmax=col_range,
)

axes[1, 1].scatter(
    x_C[cc],
    y_C[cc],
    c=1e6 * (phase_m2_new_C2[cc] - (meas_C[cc])) / tele_geo.k,
    vmin=-col_range,
    vmax=col_range,
)

axes[2, 0].scatter(
    x_C[cc],
    y_C[cc],
    c=1e6 * (phase_m1_new_C3[cc] - (meas_C[cc])) / tele_geo.k,
    vmin=-col_range,
    vmax=col_range,
)

axes[2, 1].scatter(
    x_C[cc],
    y_C[cc],
    c=1e6 * (phase_m2_new_C3[cc] - (meas_C[cc])) / tele_geo.k,
    vmin=-col_range,
    vmax=col_range,
)
axes[0, 0].set_aspect("equal")
axes[0, 1].set_aspect("equal")
axes[1, 0].set_aspect("equal")
axes[1, 1].set_aspect("equal")
axes[2, 0].set_aspect("equal")
axes[2, 1].set_aspect("equal")
axes[0, 0].set_yticklabels([])
axes[0, 0].set_xticklabels([])
plt.tight_layout(w_pad=None, h_pad=None)
fig.colorbar(first, ax=axes)
plt.show()

## RMS Across the Focal  Plane
We now want to study how well the ML model corrects for surface defects across the entire focal plane, not only where we put the receivers. To do this, we repeat the above calculations over a grid in the focal plane for singular, binocular, and trinocular sets. 

In [ ]:
foc_x = np.linspace(-1000, 1000, 11) # sweep in focal plane [mm]
foc_z = np.linspace(-1000, 1000, 11) # sweep in focal plane [mm]

foc_el = [oa.el_offset(a) for a in foc_z]
foc_az = [oa.az_offset(a) for a in foc_x]

shift_x = [oa.sh_x(a) for a in foc_x]
shift_z = [oa.sh_z(a) for a in foc_z]

focal_plane_rms1 = []
focal_plane_rms2 = []
focal_plane_rms3 = []

for ii in range(len(foc_x)):
    for jj in range(len(foc_z)):

        if foc_x[ii] == 0 and foc_z[jj] == 0:
            shift = ["x", 0]
            az_temp = 0
            el_temp = 0
        elif (foc_x[ii] == 0) and (foc_z[jj] != 0):
            shift = ["y", shift_z[jj]]
            az_temp = 0
            el_temp = foc_el[jj]
        elif (foc_x[ii] != 0) and (foc_z[jj] == 0):
            shift = ["x", shift_x[ii]]
            az_temp = foc_az[ii]
            el_temp = 0
        elif (foc_x[ii] != 0) and (foc_z[jj] != 0):
            shift = ["xy", shift_x[ii], shift_z[jj]]
            az_temp = foc_az[ii]
            el_temp = foc_el[jj]

        rx = np.array([foc_x[ii], 209.09, foc_z[jj]])
        tele_geo = tg.tele_geo_init(rx[0], rx[1], rx[2], el_temp, az_temp)
        rxmirror = af.ray_mirror_pts(rx, tele_geo, th, ph)
        dat_temp = afit.take_measurement(
            np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror
        )
        dat_temp = np.loadtxt(dat_temp)
        x_temp, y_temp, meas_temp, ampl_temp, geo = afit.analyze_holography(
            dat_temp, tele_geo, 0, 1, 0, shift
        )

        meas_temp = np.where(
            (abs(ampl_temp) / np.max(abs(ampl_temp))) >= 0.3,
            meas_temp - np.mean(meas_temp),
            0,
        )

        phase_temp2 = af.model_of_adj_offs(
            (adjs_real - adj_fit2[0]) * 1e3, shift, tele_geo, "total"
        )
        focal_plane_rms2.append(
            oa.rms(
                x_temp,
                y_temp,
                1e6
                * (
                    phase_temp2
                    - np.mean(phase_temp2)
                    - (meas_temp - np.mean(meas_temp))
                )
                / tele_geo.k,
            )
        )
        phase_temp1 = af.model_of_adj_offs(
            (adjs_real - adj_fit1[0]) * 1e3, shift, tele_geo, "total"
        )
        focal_plane_rms1.append(
            oa.rms(
                x_temp,
                y_temp,
                1e6
                * (
                    phase_temp1
                    - np.mean(phase_temp1)
                    - (meas_temp - np.mean(meas_temp))
                )
                / tele_geo.k,
            )
        )
        phase_temp3 = af.model_of_adj_offs(
            (adjs_real - adj_fit3[0]) * 1e3, shift, tele_geo, "total"
        )
        focal_plane_rms3.append(
            oa.rms(
                x_temp,
                y_temp,
                1e6
                * (
                    phase_temp3
                    - np.mean(phase_temp3)
                    - (meas_temp - np.mean(meas_temp))
                )
                / tele_geo.k,
            )
        )
        print(
            rx,
            oa.rms(
                x_temp,
                y_temp,
                1e6
                * (
                    phase_temp3
                    - np.mean(phase_temp3)
                    - (meas_temp - np.mean(meas_temp))
                )
                / tele_geo.k,
            ),
        )

x_foc = np.zeros((len(foc_x), len(foc_z)))
y_foc = np.zeros((len(foc_x), len(foc_z)))
z_foc2 = np.zeros((len(foc_x), len(foc_z)))
z_foc1 = np.zeros((len(foc_x), len(foc_z)))
z_foc3 = np.zeros((len(foc_x), len(foc_z)))

kk = 0
for ii in range(len(foc_x)):
    for jj in range(len(foc_z)):
        x_foc[ii, jj] = foc_x[ii]
        y_foc[ii, jj] = foc_z[jj]
        z_foc1[ii, jj] = focal_plane_rms1[kk]
        z_foc2[ii, jj] = focal_plane_rms2[kk]
        z_foc3[ii, jj] = focal_plane_rms3[kk]
        kk += 1

plt.rcParams["image.cmap"] = "Blues"
matplotlib.rcParams["font.size"] = 28
X = x_foc
Z = y_foc

from scipy import interpolate

x = X[:, 0]
y = Z[0, :]

xx = X
yy = Z
z1 = z_foc1
z2 = z_foc2
z3 = z_foc3

f1 = interpolate.interp2d(x, y, z1, kind="cubic")
f2 = interpolate.interp2d(x, y, z2, kind="cubic")
f3 = interpolate.interp2d(x, y, z3, kind="cubic")

xnew = np.linspace(-1000, 1000, 100)
ynew = np.linspace(-1000, 1000, 100)
Xnew, Znew = np.meshgrid(xnew, ynew)

znew1 = f1(xnew, ynew)
znew2 = f2(xnew, ynew)
znew3 = f3(xnew, ynew)

z_foc3_new = np.where((Xnew / 1e3) ** 2 + (Znew / 1e3) ** 2 <= 1, znew3, np.nan)
z_foc2_new = np.where((Xnew / 1e3) ** 2 + (Znew / 1e3) ** 2 <= 1, znew2, np.nan)
z_foc1_new = np.where((Xnew / 1e3) ** 2 + (Znew / 1e3) ** 2 <= 1, znew1, np.nan)

plt.rcParams["image.cmap"] = "Blues"
matplotlib.rcParams["font.size"] = 28

m = 13

levels = np.linspace(0, 10, 5 + 1)
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(3.5, 8.4), sharex=True)
first = axes[2].contourf(Znew / 1e3, Xnew / 1e3, z_foc3_new, levels=levels)
axes[2].plot(rx_x_tri[:] / 1e3, rx_z_tri[:] / 1e3, ".", markersize=m, color="r")
axes[2].axis("equal")
axes[1].contourf(Znew / 1e3, Xnew / 1e3, z_foc2_new, levels=levels)
axes[1].plot(rx_x[0:2] / 1e3, rx_z[0:2] / 1e3, ".", markersize=m, color="r")
axes[1].axis("equal")
axes[0].contourf(Znew / 1e3, Xnew / 1e3, z_foc1_new, vmin=0, vmax=10)
axes[0].plot(rx_x[2] / 1e3, rx_z[2] / 1e3, ".", markersize=m, color="r")
axes[0].axis("equal")
axes[0].set_yticklabels([])
axes[1].set_yticklabels([])
axes[2].set_yticklabels([])
axes[0].set_xticklabels([])
plt.tight_layout(w_pad=None)

fig.colorbar(first, ax=axes)
# plt.savefig('../../../../Figs/rms_focal.pdf',bbox_inches = 'tight')
plt.show()